In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# function for comparing different dataset approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# Function for comparing different models
def score_model(model, X_train, X_valid, y_train, y_valid):
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [ ]:
estimator_vals = np.arange(50, 450, 50)
results = {}
for val in estimator_vals:
    results[val] = val

print(results)

In [ ]:
# Read the data
X_full = pd.read_csv('./data/housing_prices_competition/train.csv', index_col='Id')
X_test_full = pd.read_csv('./data/housing_prices_competition/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                  X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

print(f'Dataset shape: {X_train.shape}')
print(f'Categorical columns: \n {categorical_cols}')
print(f'Numerical columns: \n {numerical_cols}')

In [ ]:
X_train.head()

### Cross-validation scoring

In [ ]:
my_pipeline = Pipeline(steps=[
    ('preprocessor', SimpleImputer()),
    ('model', RandomForestRegressor(n_estimators=50, random_state=0))
])

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

def get_score(n_estimators):
    """Return the average MAE over 3 CV folds of random forest model.
    
    Keyword argument:
    n_estimators -- the number of trees in the forest
    """
    my_pipeline = Pipeline(steps=[
    ('preprocessor', SimpleImputer()),
    ('model', RandomForestRegressor(n_estimators, random_state=0))
    ])

    # Multiply by -1 since sklearn calculates *negative* MAE
    scores = -1 * cross_val_score(my_pipeline, X, y,
                                  cv=3,
                                  scoring='neg_mean_absolute_error')
    return scores.mean()

### Data exploration for pre-processing steps

In [ ]:
# Find the columns that have missing values
missing_val_count_by_column = (X_train.isnull().sum())
cols_with_missing_data = [col for col in X_train.columns
                          if X_train[col].isnull().any()]
cols_with_missing_numerical_data = [col for col in X_train.columns 
                                    if X_train[col].isnull().any()
                                    and X_train[col].dtype in ['int64', 'float64']]

print(missing_val_count_by_column[missing_val_count_by_column != 0])
print(f'{cols_with_missing_numerical_data=}')

# So there's missing garage data for houses with no garages. 
# Assumption: no garage is closer to a poor, old garage than to a new, good garage. 
#             so we'll input the worst metrics for the garages that don't exist.
garage_cols = [col for col in X_train.columns
               if 'Garage' in col]

print(garage_cols)

In [ ]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))

In [ ]:
clf

In [ ]:
# Define columns to handle specially 
special_cols = ['GarageYrBlt', 'LotFrontage']
numerical_cols = [col for col in numerical_cols if col not in special_cols]

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='mean')
garage_num_transformer = SimpleImputer(strategy='constant', fill_value=X_train['GarageYrBlt'].min())
lot_frontage_num_transformer = SimpleImputer(strategy='constant', fill_value=0)

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('garage_year', garage_num_transformer, ['GarageYrBlt']),
        ('lot_frontage', lot_frontage_num_transformer, ['LotFrontage']),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestRegressor(n_estimators=200, criterion='poisson', random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))

### Train the model on the full dataset and make predictions on the test set

In [ ]:
X_all = X_full[my_cols].copy()
clf.fit(X_all, y)
preds = clf.predict(X_all)
print('MAE:', mean_absolute_error(y, preds))

#### Explore the data

In [ ]:
# Shape of training data (num_rows, num_columns)
print(f'Dataset shape: {X_train.shape}\n')

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
cols_with_missing_data = [col for col in X_train.columns
                          if X_train[col].isnull().any()]
cols_with_categ_data = [col for col in X_train.columns
                        if X_train[col].dtype == 'object']
cols_with_safe_categ_data = [col for col in X_train.columns 
                             if set(X_valid[col]).issubset(X_train[col])]
cols_with_bad_categ_data = list(set(cols_with_categ_data) - set(cols_with_safe_categ_data))

cols_with_missing_data_percs = (missing_val_count_by_column/X_train.shape[0]*100)[missing_val_count_by_column != 0]
cols_with_majority_missing = cols_with_missing_data_percs[cols_with_missing_data_percs > 50.0]

cols_to_delete = list(set(cols_with_majority_missing.index).union(set(cols_with_bad_categ_data)))

print(f'{(missing_val_count_by_column != 0).sum()}/{X_train.shape[1]} columns with missing values')
print(f'{missing_val_count_by_column.sum()}/{X_train.size} missing data points')
print(f'Columns with missing data: {cols_with_missing_data}')
print(f'Columns with > 50 % missing data\n{cols_with_majority_missing}')
print(f'{len(cols_with_categ_data)}/{X_train.shape[1]} columns with categorical data')
print(f'Columns with categorical data: {cols_with_categ_data}')
print(f'Columns to delete: {cols_to_delete}')

# Drop columns with > 50 % of data missing or categorical data in valid set but not in train set
X_train = X_train.drop(columns = cols_to_delete)
X_valid = X_valid.drop(columns = cols_to_delete)

# Update the list of columns with categorical data
cols_with_categ_data = list(set(cols_with_categ_data) - set(cols_to_delete))

#### Ordinal and one hot encoding

In [ ]:
# Make copies 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[cols_with_categ_data] = ordinal_encoder.fit_transform(X_train[cols_with_categ_data])
label_X_valid[cols_with_categ_data] = ordinal_encoder.transform(X_valid[cols_with_categ_data])

In [ ]:
print("MAE from ordinal encoding all categorical variables: ") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

In [ ]:
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

In [ ]:
# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [ ]:
# Define the models
model_1 = RandomForestRegressor(n_estimators=50, random_state=0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
model_3 = RandomForestRegressor(n_estimators=100, criterion='absolute_error', random_state=0)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)

models = [model_1, model_2, model_3, model_4, model_5]

mae_list = []
for i in range(0, len(models)):
    mae = score_model(models[i])
    print("Model %d MAE: %d" % (i+1, mae))
    mae_list.append(mae)

best_model = models[mae_list.index(min(mae_list))]

# Fit best model on whole dataset
best_model = best_model.fit(X, y)
y_hat = best_model.predict(X_test)
mean_error = mean_absolute_error(y_test, y_hat)
error_frac = mean_error/y.mean()*100
print(f'{mean_error=:,.0f}')
print(f'{error_frac=:.0f}%')